# Data Sience Project - Task 4

### Spark with Clustering (task 3 in spark)

In [1]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('clustering_in_spark').getOrCreate()

#### Read data from csv

In [3]:
df=spark.read.csv('Hotels_data_Changed.csv',inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Snapshot ID: integer (nullable = true)
 |-- Snapshot Date: timestamp (nullable = true)
 |-- Checkin Date: timestamp (nullable = true)
 |-- Days: integer (nullable = true)
 |-- Original Price: integer (nullable = true)
 |-- Discount Price: integer (nullable = true)
 |-- Discount Code: integer (nullable = true)
 |-- Available Rooms: integer (nullable = true)
 |-- Hotel Name: string (nullable = true)
 |-- Hotel Stars: integer (nullable = true)
 |-- DayDiff: integer (nullable = true)
 |-- WeekDay: string (nullable = true)
 |-- DiscountDiff: integer (nullable = true)
 |-- DiscountPerc: double (nullable = true)



## 4.a

In [5]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DateType, IntegerType
from pyspark.sql.functions import desc

#### Get the 150 hotels with maximum records

In [6]:
# group by hotel name and count,with sortig 
hotel_names = df.groupBy('Hotel Name').count().sort(desc('count')).limit(150)

In [7]:
hotel_names.show()

+--------------------+-----+
|          Hotel Name|count|
+--------------------+-----+
|Newark Liberty In...| 5346|
|Hilton Garden Inn...| 4892|
|Residence Inn New...| 4314|
|Westin New York a...| 3792|
|Loews Regency New...| 3617|
|    Viceroy New York| 3565|
|Four Seasons Hote...| 3243|
|Langham Place New...| 3203|
|The Carlyle A Ros...| 3078|
|DoubleTree by Hil...| 2866|
|Magnuson Conventi...| 2862|
|Hilton Garden Inn...| 2822|
|Hilton Garden Inn...| 2772|
|     Conrad New York| 2677|
|Wyndham Garden Br...| 2599|
|Hilton Newark Air...| 2524|
|Omni Berkshire Place| 2358|
| Hilton Times Square| 2273|
| Park Hyatt New York| 2226|
|Homewood Suites b...| 2211|
+--------------------+-----+
only showing top 20 rows



#### Get list of the first (max) 150 hotels

In [8]:
maxHotel_names = hotel_names.toPandas()['Hotel Name'].tolist()

In [9]:
maxHotel_names

['Newark Liberty International Airport Marriott',
 'Hilton Garden Inn Times Square',
 'Residence Inn Newark Elizabeth Liberty International Airport',
 'Westin New York at Times Square',
 'Loews Regency New York Hotel',
 'Viceroy New York',
 'Four Seasons Hotel New York',
 'Langham Place New York Fifth Avenue',
 'The Carlyle A Rosewood Hotel',
 'DoubleTree by Hilton Metropolitan - New York City',
 'Magnuson Convention Center Hotel',
 'Hilton Garden Inn New York West 35th Street',
 'Hilton Garden Inn New York-Times Square Central',
 'Conrad New York',
 'Wyndham Garden Brooklyn Sunset Park',
 'Hilton Newark Airport',
 'Omni Berkshire Place',
 'Hilton Times Square',
 'Park Hyatt New York',
 'Homewood Suites by Hilton NY Midtown Manhattan Times Square',
 'Grand Hyatt New York',
 'The Plaza Hotel',
 'Quality Inn Woodside',
 'Hyatt Union Square New York',
 'Le Parker Meridien New York',
 'The New York EDITION',
 'W New York - Union Square',
 'Renaissance Newark Airport Hotel',
 'Hampton Inn N

#### Adding and filtering the other features

In [10]:
hotels_data = df.filter(col('Hotel Name').isin(maxHotel_names))

#### Check

In [11]:
hotels_data

DataFrame[_c0: int, Snapshot ID: int, Snapshot Date: timestamp, Checkin Date: timestamp, Days: int, Original Price: int, Discount Price: int, Discount Code: int, Available Rooms: int, Hotel Name: string, Hotel Stars: int, DayDiff: int, WeekDay: string, DiscountDiff: int, DiscountPerc: double]

In [12]:
hotels_data.show()

+---+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+-------+---------+------------+------------------+
|_c0|Snapshot ID|      Snapshot Date|       Checkin Date|Days|Original Price|Discount Price|Discount Code|Available Rooms|          Hotel Name|Hotel Stars|DayDiff|  WeekDay|DiscountDiff|      DiscountPerc|
+---+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+-------+---------+------------+------------------+
|  2|          1|2015-07-17 00:00:00|2015-08-13 00:00:00|   5|          4370|          4240|            1|              3|The Peninsula New...|          5|     27| Thursday|         130|2.9748283752860414|
|  3|          1|2015-07-17 00:00:00|2015-07-26 00:00:00|   5|          1739|          1667|            1|             18|Eventi Hotel a Ki...|          4|      9|   Sunday|   

## 4.b

#### Get the 40 hotels with maximum checkin dates

In [13]:
# group by checkin and count, with sorting
checkin = hotels_data.groupBy('Checkin Date').count().sort(desc('count')).limit(40)

In [14]:
checkin.show()

+-------------------+-----+
|       Checkin Date|count|
+-------------------+-----+
|2015-11-11 00:00:00| 2302|
|2015-10-14 00:00:00| 1887|
|2015-11-04 00:00:00| 1885|
|2015-08-19 00:00:00| 1883|
|2015-10-28 00:00:00| 1861|
|2015-10-21 00:00:00| 1817|
|2015-11-06 00:00:00| 1808|
|2015-08-12 00:00:00| 1765|
|2015-11-05 00:00:00| 1684|
|2015-10-22 00:00:00| 1662|
|2015-11-12 00:00:00| 1649|
|2015-10-29 00:00:00| 1623|
|2015-09-10 00:00:00| 1623|
|2015-09-09 00:00:00| 1616|
|2015-11-18 00:00:00| 1582|
|2015-08-26 00:00:00| 1559|
|2015-11-10 00:00:00| 1548|
|2015-11-13 00:00:00| 1547|
|2015-10-15 00:00:00| 1473|
|2015-11-21 00:00:00| 1469|
+-------------------+-----+
only showing top 20 rows



#### get list of the first (max) 40 checkin dates

In [15]:
maxCheckin = checkin.toPandas()['Checkin Date'].tolist()

In [16]:
maxCheckin

[Timestamp('2015-11-11 00:00:00'),
 Timestamp('2015-10-14 00:00:00'),
 Timestamp('2015-11-04 00:00:00'),
 Timestamp('2015-08-19 00:00:00'),
 Timestamp('2015-10-28 00:00:00'),
 Timestamp('2015-10-21 00:00:00'),
 Timestamp('2015-11-06 00:00:00'),
 Timestamp('2015-08-12 00:00:00'),
 Timestamp('2015-11-05 00:00:00'),
 Timestamp('2015-10-22 00:00:00'),
 Timestamp('2015-11-12 00:00:00'),
 Timestamp('2015-09-10 00:00:00'),
 Timestamp('2015-10-29 00:00:00'),
 Timestamp('2015-09-09 00:00:00'),
 Timestamp('2015-11-18 00:00:00'),
 Timestamp('2015-08-26 00:00:00'),
 Timestamp('2015-11-10 00:00:00'),
 Timestamp('2015-11-13 00:00:00'),
 Timestamp('2015-10-15 00:00:00'),
 Timestamp('2015-11-21 00:00:00'),
 Timestamp('2015-09-30 00:00:00'),
 Timestamp('2015-10-30 00:00:00'),
 Timestamp('2015-09-16 00:00:00'),
 Timestamp('2015-09-17 00:00:00'),
 Timestamp('2015-11-28 00:00:00'),
 Timestamp('2015-10-01 00:00:00'),
 Timestamp('2015-11-26 00:00:00'),
 Timestamp('2015-09-11 00:00:00'),
 Timestamp('2015-09-

#### Adding and filtering the other features

In [17]:
hotelsCheckin = hotels_data.filter(col('Checkin Date').isin(maxCheckin))

#### Check

In [18]:
hotelsCheckin

DataFrame[_c0: int, Snapshot ID: int, Snapshot Date: timestamp, Checkin Date: timestamp, Days: int, Original Price: int, Discount Price: int, Discount Code: int, Available Rooms: int, Hotel Name: string, Hotel Stars: int, DayDiff: int, WeekDay: string, DiscountDiff: int, DiscountPerc: double]

In [19]:
hotelsCheckin.show()

+---+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+-------+---------+------------+------------------+
|_c0|Snapshot ID|      Snapshot Date|       Checkin Date|Days|Original Price|Discount Price|Discount Code|Available Rooms|          Hotel Name|Hotel Stars|DayDiff|  WeekDay|DiscountDiff|      DiscountPerc|
+---+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+-------+---------+------------+------------------+
|  2|          1|2015-07-17 00:00:00|2015-08-13 00:00:00|   5|          4370|          4240|            1|              3|The Peninsula New...|          5|     27| Thursday|         130|2.9748283752860414|
|  4|          1|2015-07-17 00:00:00|2015-08-12 00:00:00|   5|          1739|          1672|            1|              3|Eventi Hotel a Ki...|          4|     26|Wednesday|   

## 4.c

#### We need to take for each checkin the 4 prices for the 4 discount codes, for this we need to make a new table and do join with the table that we already have

In [20]:
#the unique hotels name
names=hotelsCheckin.select('Hotel Name').distinct().collect()

In [21]:
#the unique hotels name- list
namesList= [(row['Hotel Name']) for row in names]

In [22]:
namesList

['Bentley Hotel',
 'The Westin New York Grand Central',
 'Westin New York at Times Square',
 'Super 8 Brooklyn   Park Slope Hotel',
 'Four Seasons Hotel New York',
 'Omni Berkshire Place',
 'DoubleTree by Hilton Metropolitan - New York City',
 'Dumont NYC-an Affinia hotel',
 'Hampton Inn Manhattan Downtown-Financial District',
 'Eventi Hotel a Kimpton Hotel',
 'Magnuson Convention Center Hotel',
 'Courtyard Newark Elizabeth',
 'Park Hyatt New York',
 'Roxy Hotel Tribeca (formerly the Tribeca Grand Hotel)',
 'The Kitano New York',
 'Waldorf Astoria New York',
 'Courtyard by Marriott New York Manhattan SoHo',
 'Hyatt Place New York Midtown South',
 'Hilton Garden Inn New York Manhattan-Chelsea',
 'Baccarat Hotel and Residences New York',
 'The St. Regis New York',
 'Hampton Inn Manhattan Times Square South',
 'New York Hilton Midtown',
 'The Pearl New York',
 'Best Western Plus Newark Airport West',
 'Wyndham Garden - Manhattan Chelsea West',
 'Martha Washington',
 'The Plaza Hotel',
 'G

In [23]:
#the unique checking date
checkins=hotelsCheckin.select('Checkin Date').distinct().collect()

In [24]:
#the unique checking date- list
checkinsList= [(row['Checkin Date']) for row in checkins]

In [25]:
checkinsList

[datetime.datetime(2015, 11, 26, 0, 0),
 datetime.datetime(2015, 8, 26, 0, 0),
 datetime.datetime(2015, 10, 15, 0, 0),
 datetime.datetime(2015, 9, 16, 0, 0),
 datetime.datetime(2015, 9, 11, 0, 0),
 datetime.datetime(2015, 8, 28, 0, 0),
 datetime.datetime(2015, 11, 5, 0, 0),
 datetime.datetime(2015, 8, 12, 0, 0),
 datetime.datetime(2015, 11, 10, 0, 0),
 datetime.datetime(2015, 11, 13, 0, 0),
 datetime.datetime(2015, 10, 1, 0, 0),
 datetime.datetime(2015, 8, 27, 0, 0),
 datetime.datetime(2015, 11, 3, 0, 0),
 datetime.datetime(2015, 9, 30, 0, 0),
 datetime.datetime(2015, 8, 19, 0, 0),
 datetime.datetime(2015, 11, 11, 0, 0),
 datetime.datetime(2015, 11, 7, 0, 0),
 datetime.datetime(2015, 9, 9, 0, 0),
 datetime.datetime(2015, 11, 25, 0, 0),
 datetime.datetime(2015, 11, 4, 0, 0),
 datetime.datetime(2015, 10, 27, 0, 0),
 datetime.datetime(2015, 10, 29, 0, 0),
 datetime.datetime(2015, 10, 30, 0, 0),
 datetime.datetime(2015, 10, 21, 0, 0),
 datetime.datetime(2015, 10, 28, 0, 0),
 datetime.datet

In [26]:
#the unique discount code
uniqueCode= [1,2,3,4]

#### Build the table that will help us insert the 4 codes for each checkin

In [27]:
helpTable=[]
for x in namesList:
    for y in checkinsList:
        for z in uniqueCode:
            helpTable.append([x, y ,z,-1])

In [28]:
helpTable

[['Bentley Hotel', datetime.datetime(2015, 11, 26, 0, 0), 1, -1],
 ['Bentley Hotel', datetime.datetime(2015, 11, 26, 0, 0), 2, -1],
 ['Bentley Hotel', datetime.datetime(2015, 11, 26, 0, 0), 3, -1],
 ['Bentley Hotel', datetime.datetime(2015, 11, 26, 0, 0), 4, -1],
 ['Bentley Hotel', datetime.datetime(2015, 8, 26, 0, 0), 1, -1],
 ['Bentley Hotel', datetime.datetime(2015, 8, 26, 0, 0), 2, -1],
 ['Bentley Hotel', datetime.datetime(2015, 8, 26, 0, 0), 3, -1],
 ['Bentley Hotel', datetime.datetime(2015, 8, 26, 0, 0), 4, -1],
 ['Bentley Hotel', datetime.datetime(2015, 10, 15, 0, 0), 1, -1],
 ['Bentley Hotel', datetime.datetime(2015, 10, 15, 0, 0), 2, -1],
 ['Bentley Hotel', datetime.datetime(2015, 10, 15, 0, 0), 3, -1],
 ['Bentley Hotel', datetime.datetime(2015, 10, 15, 0, 0), 4, -1],
 ['Bentley Hotel', datetime.datetime(2015, 9, 16, 0, 0), 1, -1],
 ['Bentley Hotel', datetime.datetime(2015, 9, 16, 0, 0), 2, -1],
 ['Bentley Hotel', datetime.datetime(2015, 9, 16, 0, 0), 3, -1],
 ['Bentley Hotel'

#### Making the schema for dataframe

In [29]:
from pyspark.sql.types import *

In [30]:
dfSchema = StructType([StructField("Hotel Name", StringType()),StructField("Checkin Date", StringType()),StructField("Discount Code",  IntegerType()),StructField("Discount Price", LongType())])

#### Creating spark dataframe

In [31]:
s_df= spark.createDataFrame(helpTable, schema=dfSchema)

In [55]:
s_df.show()

+-------------+--------------------+-------------+--------------+
|   Hotel Name|        Checkin Date|Discount Code|Discount Price|
+-------------+--------------------+-------------+--------------+
|Bentley Hotel|java.util.Gregori...|            1|            -1|
|Bentley Hotel|java.util.Gregori...|            2|            -1|
|Bentley Hotel|java.util.Gregori...|            3|            -1|
|Bentley Hotel|java.util.Gregori...|            4|            -1|
|Bentley Hotel|java.util.Gregori...|            1|            -1|
|Bentley Hotel|java.util.Gregori...|            2|            -1|
|Bentley Hotel|java.util.Gregori...|            3|            -1|
|Bentley Hotel|java.util.Gregori...|            4|            -1|
|Bentley Hotel|java.util.Gregori...|            1|            -1|
|Bentley Hotel|java.util.Gregori...|            2|            -1|
|Bentley Hotel|java.util.Gregori...|            3|            -1|
|Bentley Hotel|java.util.Gregori...|            4|            -1|
|Bentley H

#### Filter from dataframe that we already have only :Hotel Name, Checkin Date, Discount Code, Discount Price

In [33]:
hotels_df=hotelsCheckin.select('Hotel Name', 'Checkin Date','Discount Code', 'Discount Price')

#### Union the dataframe with the table that we build

In [34]:
all_df=hotels_df.union(s_df)

In [35]:
all_df.show()

+--------------------+-------------------+-------------+--------------+
|          Hotel Name|       Checkin Date|Discount Code|Discount Price|
+--------------------+-------------------+-------------+--------------+
|The Peninsula New...|2015-08-13 00:00:00|            1|          4240|
|Eventi Hotel a Ki...|2015-08-12 00:00:00|            1|          1672|
|Grand Hyatt New York|2015-08-12 00:00:00|            1|          1183|
|Grand Hyatt New York|2015-08-13 00:00:00|            1|          1201|
| Park Hyatt New York|2015-08-12 00:00:00|            1|          3378|
| Park Hyatt New York|2015-08-13 00:00:00|            1|          3678|
|Loews Regency New...|2015-08-13 00:00:00|            1|          1795|
|Hilton Garden Inn...|2015-08-19 00:00:00|            1|           671|
|Hilton Newark Air...|2015-08-12 00:00:00|            1|           690|
|Hilton Newark Air...|2015-08-19 00:00:00|            1|           690|
|Newark Liberty In...|2015-08-12 00:00:00|            1|        

#### Group by to order the details

In [36]:
all_df = all_df.groupBy('Hotel name','Checkin Date', 'Discount Code').min('Discount Price')

In [37]:
all_df.show()

+--------------------+-------------------+-------------+-------------------+
|          Hotel name|       Checkin Date|Discount Code|min(Discount Price)|
+--------------------+-------------------+-------------+-------------------+
|The Lexington New...|2015-09-16 00:00:00|            1|               1648|
|Hilton Garden Inn...|2015-09-18 00:00:00|            2|               1275|
|    Viceroy New York|2015-09-17 00:00:00|            4|               2506|
|Le Parker Meridie...|2015-09-17 00:00:00|            3|               2044|
|New York Marriott...|2015-09-10 00:00:00|            2|               1615|
|Fairfield Inn by ...|2015-09-18 00:00:00|            2|                935|
|McCarren Hotel & ...|2015-09-09 00:00:00|            2|               2219|
|Hilton Garden Inn...|2015-09-30 00:00:00|            2|               1669|
|        Aloft Harlem|2015-09-10 00:00:00|            1|               1065|
|    Viceroy New York|2015-10-07 00:00:00|            2|               2455|

#### Partition data by hotel name

In [38]:
all_dfparts=all_df.repartition("Hotel name")

## 4.d

#### Split to two groups:  with price  -1,  without -1(the others)

In [39]:
groupMinus = all_df.filter(col('min(Discount Price)') == -1)
groupWithout = all_df.filter(col('min(Discount Price)') > -1)

### Normalization

In [40]:
groupWithout.createOrReplaceTempView("normalization")

#### Creating SQLContext for SQL converting

In [42]:
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
import pyspark.sql.functions as func

In [43]:
sqlContext = SQLContext(spark)

#### Preparing data for normalization

In [44]:
dataFrame = sqlContext.table("normalization")

In [45]:
#Partioning data for groups so we can applay function on groups
windowSpec = Window.partitionBy(groupWithout['Hotel name']) 

### Normalize function

In [46]:
normalize =  ((dataFrame['min(Discount Price)'] - func.min(dataFrame['min(Discount Price)']).over(windowSpec)) / (func.max(dataFrame['min(Discount Price)']).over(windowSpec) - func.min(dataFrame['min(Discount Price)']).over(windowSpec)) * 100)

In [47]:
normalized = dataFrame.select(
  dataFrame['Hotel Name'],
  dataFrame['Checkin Date'],
  dataFrame['Discount Code'],
  normalize.alias("Normal"))

#### Getting back the values -1

In [48]:
#Changing column name for same schema
groupMinus = groupMinus.withColumnRenamed('min(Discount Price)','Normal')

#### Union all data frames with sorting

In [49]:
normalized = normalized.union(groupMinus)
normalized = normalized.orderBy(['Hotel name','Checkin Date','Discount Code'])

In [50]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [51]:
normal_toList = normalized.groupBy('Hotel name').agg(F.collect_list('Normal').alias("Normal"))

In [52]:
#parsing to df 
all_hotels =  normal_toList.select([normal_toList["Hotel name"]] +  [normal_toList.Normal[i] for i in range(160)])

## 4.e

#### We received dataframe that each line represents one hotel and has 161 columns - one for the hotel name and 160 for the normalized prices

In [53]:
all_hotels.show()

+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------

## 4.f